In [17]:
!pip install git+https://github.com/sjsu2024-data298-team6/ultralytics.git
!pip install boto3 tensorflowjs psycopg2-binary

import boto3
import botocore
from botocore.exceptions import ClientError
import os
from google.colab import userdata
import shutil
from pathlib import Path


os.environ['AWS_ACCESS_KEY_ID'] = userdata.get("AWS_ACCESS_KEY_ID")
os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get("AWS_SECRET_ACCESS_KEY")
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

  Cloning https://github.com/sjsu2024-data298-team6/ultralytics.git to /tmp/pip-req-build-d89_v020
  Running command git clone --filter=blob:none --quiet https://github.com/sjsu2024-data298-team6/ultralytics.git /tmp/pip-req-build-d89_v020
  Resolved https://github.com/sjsu2024-data298-team6/ultralytics.git to commit 02ff417f1b3e42563298f366ef7587519fb63124
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [18]:
import psycopg2
from urllib.parse import urlparse

DATABASE_URL = userdata.get("D298_DB_URI")

def execute_query(query, data=None, fetch=False):
    try:
        result = urlparse(DATABASE_URL)
        db_user = result.username
        db_password = result.password
        db_name = result.path[1:]
        db_host = result.hostname
        db_port = result.port

        with psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port
        ) as connection:
            with connection.cursor() as cursor:
                cursor.execute(query, data)
                if fetch:
                    results = cursor.fetchall()
                    return results
                else:
                    connection.commit()
                    return None

    except Exception as e:
        print(f"Error: {e}")
        return None

In [19]:
query = """SELECT * FROM "ModelResults" where "isActive"=True and "datasetId"=30 and "tfjsS3Key"='';"""
results = execute_query(query, fetch=True)
models = []
if results:
    for row in results:
        if "manual_entry" in row[9]:
            continue
        models.append([row[0],row[11]])
else:
    print("No results found or error occurred.")
models

[[63, 'runs/rtdetr_base_1746512165_weights.pt'],
 [75, 'runs/yolov8_mhsa_1746790519_weights.pt']]

In [20]:
from ultralytics import YOLO, RTDETR

In [21]:
!rm -rf custom* model* yolo*

In [22]:
models

[[63, 'runs/rtdetr_base_1746512165_weights.pt'],
 [75, 'runs/yolov8_mhsa_1746790519_weights.pt']]

In [23]:
def printt(*args, **kwargs):
    print("-" * 100, end="")
    print(*args, **kwargs)

In [24]:
s3 = boto3.resource("s3")
BUCKET = "sjsu-fall24-data298-team6-bucket"

for m in models:
    printt()
    printt()
    # --- PREPARATION ---
    model_name = m[1].split("/")[1].split(".")[0]
    tfjs_folder_name = f"{model_name}_tfjs"
    model_file_name = "model.pt"

    # Clean up previous files
    !rm -rf model.pt model.onnx model.pb model_saved_model/

    key = m[1]
    id = m[0]

    # --- DOWNLOAD ---
    try:
        s3.Bucket(BUCKET).download_file(key, model_file_name)
        printt(f"Downloaded {key} from S3 to {model_file_name}")
    except Exception as e:
        printt(f"Error downloading {key}: {e}")
        continue  # Skip to the next model

    # --- MODEL LOADING ---
    try:
        if "rtdetr" in key:
            model = RTDETR(model_file_name)
        else:
            model = YOLO(model_file_name)
    except Exception as e:
        printt(f"Error loading model {key}: {e}")
        continue

    # --- EXPORT ---
    try:
        model.export(format="tfjs", half=True)
    except Exception as e:
        printt(f"Error exporting model {key} to TFJS: {e}")
        continue

    # --- RENAMING ---
    try:
        shutil.move("model_web_model", tfjs_folder_name)
        printt(f"Renamed model_web_model to {tfjs_folder_name}")
    except FileNotFoundError:
        printt("Error: model_web_model directory not found after export.")
        continue
    except Exception as e:
        printt(f"Error renaming directory: {e}")
        continue


----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------Downloaded runs/rtdetr_base_1746512165_weights.pt from S3 to model.pt
Ultralytics 8.3.86 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (AMD EPYC 7B12)
rtdetr-custom summary: 302 layers, 32,004,290 parameters, 0 gradients, 103.5 GFLOPs

PyTorch: starting from 'model.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 300, 14) (63.1 MB)

TensorFlow SavedModel: starting export with tensorflow 2.18.0...

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.52...
ONNX: export success ✅ 25.3s, saved as 'model.onnx' (122.9 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.26.3...

WARNING Dimensional compression of `Slice` fails. node.name: /m

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR: The trace log is below.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/onnx2tf/utils/common_functions.py", line 312, in print_wrapper_func
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/onnx2tf/utils/common_functions.py", line 385, in inverted_operation_enable_disable_wrapper_func
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/onnx2tf/utils/common_functions.py", line 55, in get_replacement_parameter_wrapper_func
    func(*args, **kwargs)
  File "/usr/local/lib/python3.11/dist-packages/onnx2tf/ops/GridSample.py", line 663, in make_node
    define_fast_gridsample(
  File "/usr/local/lib/python3.11/dist-packages/onnx2tf/ops/GridSample.py", line 181, in define_fast_gridsample
    pixs = (tf.math.multiply(grid + 1.0, tf.convert_to_tensor([w_in, h_in], dtype=tf.float32)) - 1.0) * 0.5
                          

TypeError: object of type 'NoneType' has no len()

In [ ]:
def upload_directory_to_s3(local_directory, bucket, s3_prefix):
    s3_uploader = boto3.client(
        "s3",
        region_name='us-east-1',
        aws_access_key_id=userdata.get("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=userdata.get("AWS_SECRET_ACCESS_KEY"),
    )
    for root, _, files in os.walk(local_directory):
        for filename in files:
            local_path = os.path.join(root, filename)
            relative_path = os.path.relpath(local_path, local_directory)
            s3_path = os.path.join(s3_prefix, relative_path)
            try:
                s3_uploader.upload_file(local_path, bucket, s3_path)
                print(f"Uploaded {local_path} to s3://{bucket}/{s3_path}")
            except Exception as e:
                print(f"Error uploading {local_path}: {e}")
                raise #Re-raise the exception to stop the loop


for m in models:
    print()
    print()

    key = m[1]
    id = m[0]

    model_name = m[1].split("/")[1].split(".")[0]
    tfjs_folder_name = Path(f"{model_name}_tfjs")
    is_valid = False
    if os.path.isdir(tfjs_folder_name):
        if (tfjs_folder_name / "metadata.yaml").exists() and (tfjs_folder_name / "model.json").exists():
            bin_files = list(tfjs_folder_name.glob("group*.bin"))
            if len(bin_files) > 0:
                expected_files = ({"metadata.yaml", "model.json"}|{f.name for f in bin_files})
                actual_files = set(os.listdir(tfjs_folder_name))
                if actual_files == expected_files:
                    is_valid = True

    if not is_valid:
        print(f"Skipping {tfjs_folder_name}")
        continue

    # --- UPLOAD ---
    s3_prefix = f"tfjs_models/{tfjs_folder_name}"
    try:
        upload_directory_to_s3(tfjs_folder_name, BUCKET, s3_prefix)
        print(f"Uploaded {tfjs_folder_name} to s3://{BUCKET}/{s3_prefix}")
        model_s3_key = s3_prefix
    except Exception as e:
        print(f"Error uploading {tfjs_folder_name} to S3: {e}")
        continue

    # --- DATABASE UPDATE ---
    try:
        query = 'UPDATE "ModelResults" SET "tfjsS3Key" = %s WHERE id = %s;'
        data = (model_s3_key, id)
        print(data)
        execute_query(query, data=data)
        print(f"Updated database for id {id} with S3 key {model_s3_key}")
    except Exception as e:
        print(f"Error updating database for id {id}: {e}")